In [83]:
from wrangle import wrangle_wine


# Visualize your success!
import seaborn as sns
import matplotlib.pyplot as plt

#Stats
from scipy import stats

# sklearn for modeling:
from sklearn.tree import DecisionTreeClassifier,\
export_text, \
plot_tree
from sklearn.metrics import accuracy_score, \
classification_report, \
confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Get the DF


In [42]:
train, val, test = wrangle_wine()

In [4]:
train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
2391,6.4,0.460,0.08,4.90,0.046,34.0,144.0,0.99445,3.10,0.56,10.0,5,white
989,9.7,0.295,0.40,1.50,0.073,14.0,21.0,0.99556,3.14,0.51,10.9,6,red
2805,7.0,0.260,0.46,15.55,0.037,61.0,171.0,0.99860,2.94,0.35,8.8,6,white
2909,5.2,0.365,0.08,13.50,0.041,37.0,142.0,0.99700,3.46,0.39,9.9,6,white
41,8.8,0.610,0.30,2.80,0.088,17.0,46.0,0.99760,3.26,0.51,9.3,4,red


In [43]:
conditions = [
    (train.quality == 3) | (train.quality == 4) | (train.quality == 5),
    (train.quality == 6), 
    (train.quality == 7) | (train.quality == 8) | (train.quality == 9)
]


values = ['low', 'medium', 'high']

train['quality_bins'] = np.select(conditions, values, default=np.nan)

In [44]:
conditions = [
    (val.quality == 3) | (val.quality == 4) | (val.quality == 5),
    (val.quality == 6), 
    (val.quality == 7) | (val.quality == 8) | (val.quality == 9)
]


values = ['low', 'medium', 'high']

val['quality_bins'] = np.select(conditions, values, default=np.nan)

In [45]:
conditions = [
    (test.quality == 3) | (test.quality == 4) | (test.quality == 5),
    (test.quality == 6), 
    (test.quality == 7) | (test.quality == 8) | (test.quality == 9)
]


values = ['low', 'medium', 'high']

test['quality_bins'] = np.select(conditions, values, default=np.nan)

In [23]:
train.quality_bins.value_counts()

medium    1587
low       1337
high       713
Name: quality_bins, dtype: int64

In [78]:
baseline_accuracy  = 713 / len(train)

In [79]:
baseline_accuracy

0.19604069287874623

In [46]:
x_cols = ['volatile acidity', 'chlorides', 'density', 'alcohol']
y_cols = 'quality_bins'

In [32]:
train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color,quality_bins
2391,0.194690,0.304,0.048193,0.138710,0.056856,0.114583,0.317972,0.315926,0.294574,0.191011,0.330579,5,white,low
989,0.486726,0.172,0.240964,0.029032,0.102007,0.045139,0.034562,0.363833,0.325581,0.162921,0.479339,6,red,medium
2805,0.247788,0.144,0.277108,0.482258,0.041806,0.208333,0.380184,0.495037,0.170543,0.073034,0.132231,6,white,medium
2909,0.088496,0.228,0.048193,0.416129,0.048495,0.125000,0.313364,0.425982,0.573643,0.095506,0.314050,6,white,medium
41,0.407080,0.424,0.180723,0.070968,0.127090,0.055556,0.092166,0.451877,0.418605,0.162921,0.214876,4,red,low


In [47]:
x_train = train[x_cols]
y_train = train[y_cols]

x_val = val[x_cols]
y_val = val[y_cols]

x_test = test[x_cols]
y_test = test[y_cols]

In [70]:
def scale_data(train, val, test):
    '''
    This function scales the data using the sklearn minmax scaler. Does not scale the target or categorical columns
    '''
    x_cols =  ['volatile acidity', 'chlorides', 'density', 'alcohol']
    split = [train, val, test]
    scale_list= []
    scaler = MinMaxScaler()
    scaler.fit(train[x_cols])
    for cut in split:
        cut_copy = cut.copy()
        cut_copy[x_cols] = scaler.transform(cut_copy[x_cols])
        scale_list.append(cut_copy)

    
    return scale_list[0], scale_list[1], scale_list[2] 

In [71]:
train_scaled, val_scaled, test_scaled = scale_data(train, val, test)

In [72]:
train_scaled

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color,quality_bins
2391,6.4,0.304,0.08,4.90,0.056856,34.0,144.0,0.315926,3.10,0.56,0.330579,5,white,low
989,9.7,0.172,0.40,1.50,0.102007,14.0,21.0,0.363833,3.14,0.51,0.479339,6,red,medium
2805,7.0,0.144,0.46,15.55,0.041806,61.0,171.0,0.495037,2.94,0.35,0.132231,6,white,medium
2909,5.2,0.228,0.08,13.50,0.048495,37.0,142.0,0.425982,3.46,0.39,0.314050,6,white,medium
41,8.8,0.424,0.30,2.80,0.127090,17.0,46.0,0.451877,3.26,0.51,0.214876,4,red,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2319,6.6,0.168,0.44,9.00,0.068562,62.0,178.0,0.419508,3.02,0.45,0.148760,5,white,low
2701,6.3,0.064,0.40,1.60,0.035117,59.0,148.0,0.184290,3.44,0.53,0.561983,5,white,low
6189,6.4,0.200,0.30,7.20,0.048495,42.0,168.0,0.266724,3.22,0.49,0.512397,6,white,medium
2133,7.4,0.136,0.36,13.20,0.091973,53.0,178.0,0.451877,3.01,0.48,0.165289,6,white,medium


In [57]:
def get_decisionTree_model():
    """
    Returns a decision treen model with a max depth of 5
    prints out the Accuracy of train and validate and the 
    classification report
    """
    clf = DecisionTreeClassifier(max_depth=7, random_state=706)
    #class_weight='balanced'
    # fit the thing
    clf.fit(x_train, y_train)

    model_proba = clf.predict_proba(x_train)
    model_preds = clf.predict(x_train)

    model_score = clf.score(x_train, y_train)

    #classification report:
    print(
        classification_report(y_train,
                          model_preds))
    print('Accuracy of Random Tree classifier on training set: {:.2f}'
     .format(clf.score(x_train, y_train)))
    print('Accuracy of Random Tree classifier on validation set: {:.2f}'
     .format(clf.score(x_val, y_val)))
    return clf


In [73]:
clf = get_decisionTree_model()

              precision    recall  f1-score   support

        high       0.55      0.66      0.60       713
         low       0.71      0.71      0.71      1337
      medium       0.63      0.57      0.60      1587

    accuracy                           0.64      3637
   macro avg       0.63      0.65      0.64      3637
weighted avg       0.65      0.64      0.64      3637

Accuracy of Random Tree classifier on training set: 0.64
Accuracy of Random Tree classifier on validation set: 0.58


In [49]:
baseline_accuracy

0.43634863898817705

In [60]:
def get_random_forest():
    """
    Runs through two for loops from range 1 - 5 each time increasing the max depth 
    and min sample leaf
    puts all of the models in a pandas data frame and sorts for the hightes valadation 
    Prints out the classification report on the best model
    """
    
    model_list = []

    for j in range (1, 5):
        for i in range(2, 5):
            rf = RandomForestClassifier(max_depth=i, min_samples_leaf=j, random_state=706)

            rf = rf.fit(x_train, y_train)
            train_accuracy = rf.score(x_train, y_train)
            validate_accuracy = rf.score(x_val, y_val)
            model_preds = rf.predict(x_train)

            output = {
                "min_samples_per_leaf": j,
                "max_depth": i,
                "train_accuracy": train_accuracy,
                "validate_accuracy": validate_accuracy,
                'model_preds': model_preds
            }
            model_list.append(output)
            
    df = pd.DataFrame(model_list)
    df["difference"] = df.train_accuracy - df.validate_accuracy
    df["baseline_accuracy"] = baseline_accuracy
    # df[df.validate_accuracy > df.baseline_accuracy + .05].sort_values(by=['difference'], ascending=True).head(15)
    df.sort_values(by=['validate_accuracy'], ascending=False).head(1)
    
    #classification report:
    print(classification_report(y_train, df['model_preds'][1]))
    return df.sort_values(by=['validate_accuracy'], ascending=False).head(1)

In [62]:
rf = get_random_forest()
rf

              precision    recall  f1-score   support

        high       0.56      0.30      0.39       713
         low       0.65      0.66      0.65      1337
      medium       0.53      0.63      0.57      1587

    accuracy                           0.57      3637
   macro avg       0.58      0.53      0.54      3637
weighted avg       0.58      0.57      0.57      3637



,min_samples_per_leaf,max_depth,train_accuracy,validate_accuracy,model_preds,difference,baseline_accuracy
8,3,4,0.587297,0.576282,"[low, medium, low, low, low, low, medium, medi...",0.011015,0.436349


In [75]:
x_train = train_scaled[x_cols]
y_train = train_scaled[y_cols]

x_val = val_scaled[x_cols]
y_val = val_scaled[y_cols]

x_test = test_scaled[x_cols]
y_test = test_scaled[y_cols]

In [76]:
def get_logReg_model():
    logit = LogisticRegression()
    logit.fit(x_train, y_train)
    y_pred = logit.predict(x_train)
    logit_val = logit.predict(x_val)
    print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(x_train, y_train)))
    print('Accuracy of Logistic Regression classifier on validation set: {:.2f}'
     .format(logit.score(x_val, y_val)))
    print(
    classification_report(y_train,
                      y_pred))

In [77]:
lr = get_logReg_model()

Accuracy of Logistic Regression classifier on training set: 0.56
Accuracy of Logistic Regression classifier on validation set: 0.58
              precision    recall  f1-score   support

        high       0.54      0.27      0.36       713
         low       0.63      0.65      0.64      1337
      medium       0.52      0.63      0.57      1587

    accuracy                           0.56      3637
   macro avg       0.57      0.51      0.52      3637
weighted avg       0.57      0.56      0.55      3637



In [81]:
plt.figure(figsize=(100,100))
plot_tree(clf,feature_names=x_cols)
plt.show()

In [85]:
k = 3
knn = KNeighborsClassifier(n_neighbors=k)

knn.fit(x_train, y_train)

y_pred = knn.predict(x_val)

accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 55.38%


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
